In [13]:
import requests
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd

In [14]:
user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.71 Safari/537.36'
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'
'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.83 Safari/537.1']

In [15]:
cookie={} 
# We are getting the reponse based on the search query
def getAmazonSearch(search_query):
    url="https://www.amazon.in/s?k="+search_query
    for agent in user_agents:
      header={'User-Agent': agent}
      print(url)
      page=requests.get(url,cookies=cookie,headers=header)
      if page.status_code==200:
          return page
      else:
          continue

In [16]:
# ASIN stands for Amazon Standard Identification Number.Almost every product has its own ASIN,a unique code we use to identify it.
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    for agent in user_agents:
      header={'User-Agent': agent}
      page=requests.get(url,cookies=cookie,headers=header)
      if page.status_code==200:
          return page
      else:
          continue
    return "Error"

In [18]:
# This will return the next page URL
def getnextpage(soup):
    pages = soup.find('span', {'class': 's-pagination-strip'})
    if not pages.find('span', {'class': 's-pagination-item s-pagination-next s-pagination-disabled'}):
        ref = pages.find('a', {'class': 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'})['href']
        url = 'https://www.amazon.in' + str(ref)
        return url
    else:
        return

In [23]:
response=getAmazonSearch('iphone+all+mobiles')
soup=BeautifulSoup(response.content,features="html.parser")
urls = []
while True:
    url = getnextpage(soup)
    if not url:
        break
    for agent in user_agents:
      header={'User-Agent': agent}
      response=requests.get(url,cookies=cookie,headers=header)
    soup=BeautifulSoup(response.content,features="html.parser")
    urls.append(url)

# Getting ASIN numbers for the products in the urls taken
data_asin = []
for url in urls:
  page=requests.get(url,cookies=cookie,headers=header)
  soup=BeautifulSoup(page.content,features="html.parser")
  for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"}):
     data_asin.append(i['data-asin'])
  

https://www.amazon.in/s?k=iphone+all+mobiles


In [28]:
data = {'URLS': urls}
df_url = pd.DataFrame(data)
df_url.to_csv('URLs.csv')
urls

['https://www.amazon.in/s?k=iphone+all+mobiles&page=2&qid=1707827673&ref=sr_pg_1',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=3&qid=1707827675&ref=sr_pg_2',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=4&qid=1707827676&ref=sr_pg_3',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=5&qid=1707827677&ref=sr_pg_4',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=6&qid=1707827679&ref=sr_pg_5',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=7&qid=1707827681&ref=sr_pg_6',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=8&qid=1707827683&ref=sr_pg_7',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=9&qid=1707827684&ref=sr_pg_8',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=10&qid=1707827686&ref=sr_pg_9',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=11&qid=1707827688&ref=sr_pg_10',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=12&qid=1707827689&ref=sr_pg_11',
 'https://www.amazon.in/s?k=iphone+all+mobiles&page=13&qid=1707827691&ref=sr_pg_12

In [26]:
# In this code we are scraping the Products data
link=[]
Product_title= []
Product_cost = []
Product_rating = []
Product_descrtipion = []
updated_asin = []
Total_ratings = []
print(len(data_asin))
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    print(response)
    if response == "Error":
      continue
    soup=BeautifulSoup(response.content)
    
    title = soup.find("span",{"class":'a-size-large product-title-word-break'})
    if title == None:
      continue
    price = soup.find("span",{"class":'a-offscreen'})
    if price == None:
      continue
    description_response = soup.find("ul",{"class":'a-unordered-list a-vertical a-spacing-mini'}) #Here we search initally with the parent tag
    if description_response == None:
      continue
    description = ""
    for desc in description_response.findAll("span",{"class":'a-list-item'}): #from the response of parent tag we search the children
      description += desc.get_text().strip()+"\n"
    if description == "":
      continue
    rating = soup.find("span",{"class":'a-size-medium a-color-base'})
    if rating==None:
      continue
    total_rating_response = soup.find("div",{"class":'a-row a-spacing-medium averageStarRatingNumerical'})
    if total_rating_response == None:
      continue
    total_rating = total_rating_response.find("span",{"class":'a-size-base a-color-secondary'})
    if total_rating == None:
      continue
    h_ref = soup.find("a",{'data-hook':"see-all-reviews-link-foot"})
    if h_ref == None:
      continue
    link.append(h_ref['href'])
    updated_asin.append(data_asin[i])
    Product_title.append(title.get_text().strip())
    Product_cost.append(price.get_text().strip())
    Product_rating.append(rating.get_text().strip().split()[0])
    Total_ratings.append(total_rating.get_text().strip().split()[0])
    Product_descrtipion.append(description)
    sleep(10)
    

290
https://www.amazon.in/dp/B0BDJGSK1J
<Response [200]>
https://www.amazon.in/dp/B08L5T3S7T
<Response [200]>
https://www.amazon.in/dp/B09V4MXBSN
<Response [200]>
https://www.amazon.in/dp/B0CMTWQYFM
<Response [200]>
https://www.amazon.in/dp/B0BDJ213TX
<Response [200]>
https://www.amazon.in/dp/B0BY8JZ22K
<Response [200]>
https://www.amazon.in/dp/B0C788SHHC
<Response [200]>
https://www.amazon.in/dp/B0BKZSVF27
<Response [200]>
https://www.amazon.in/dp/B0BY8MCQ9S
<Response [200]>
https://www.amazon.in/dp/B07WGPJPR3
<Response [200]>
https://www.amazon.in/dp/B0C7V7VH6Q
<Response [200]>
https://www.amazon.in/dp/B09WQYFLRX
<Response [200]>
https://www.amazon.in/dp/B0BMGB3CH9
<Response [200]>
https://www.amazon.in/dp/B0BXD5ZZQ5
<Response [200]>
https://www.amazon.in/dp/B0CKJ5ZXFK
<Response [200]>
https://www.amazon.in/dp/B082F2T5PQ
<Response [200]>
https://www.amazon.in/dp/B0C2CWV2PS
<Response [200]>
https://www.amazon.in/dp/B0BPS4YLQT
<Response [200]>
https://www.amazon.in/dp/B09SH7FDKT
<Respo

In [27]:
# Stroing ASIN, Product details and links in a dataframe. 
data = {'ASIN':updated_asin,
        'Product Title': Product_title,
        'Product Description': Product_descrtipion,
        
        }

 
df = pd.DataFrame(data)
 
print(df.head())

df.to_csv("Product_details.csv")

         ASIN                                      Product Title  \
0  B0BDJGSK1J           Apple iPhone 14 Plus (512 GB) - Midnight   
1  B08L5T3S7T                     Apple iPhone 12 (64GB) - Black   
2  B09V4MXBSN                   Apple iPhone 13 (256 GB) - Green   
3  B0CMTWQYFM  Redmi 13C (Stardust Black, 8GB RAM, 256GB Stor...   
4  B0BDJ213TX                Apple iPhone 14 (512 GB) - Midnight   

                                 Product Description  
0  16.95 cm (6.7-inch) Super Retina XDR display\n...  
1  6.1-inch (15.5 cm diagonal) Super Retina XDR d...  
2  15 cm (6.1-inch) Super Retina XDR display\nCin...  
3  Processor: High performance MediaTek G85 ; Enh...  
4  15.40 cm (6.1-inch) Super Retina XDR display\n...  
